In [ ]:
import os
import time
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1028"

import torch
torch.cuda.empty_cache()

In [ ]:
%pip install --quiet "evadb[vision,notebook]"
import evadb
cursor = evadb.connect().cursor()
"""
Issues Encountered:
cant have negative numbers in multidimensional arrays
cant have camel case words
cant have float attributes
cant change column types of udf without restarting db and deleting
query takes infinitley long when UDF in 
inner join does not work
"""

In [ ]:
### Set up Yolo UDF
cursor.query("""
            CREATE UDF IF NOT EXISTS Yolo
            TYPE  ultralytics
            'model' 'yolov8m.pt';
    """).df() 

In [ ]:
### Set up Monodepth UDF
cursor.query(""" 
            CREATE UDF IF NOT EXISTS MonodepthDetection
            IMPL'/home/youse/apperception/eva/udfs/monodepth_detection.py';
    """).df()

In [ ]:
### Set up Location UDF
cursor.query(""" 
            CREATE UDF IF NOT EXISTS LocationDetection
            IMPL'/home/youse/apperception/eva/udfs/location_detection.py';
    """).df()

In [ ]:
### Set up Intersection Query UDF
cursor.query(""" 
            CREATE UDF IF NOT EXISTS IntersectionQuery
            IMPL'/home/youse/apperception/eva/udfs/intersection_query.py';
    """).df()

In [ ]:
### Set up Relative Position Query UDF
cursor.query(""" 
            CREATE UDF IF NOT EXISTS RelativePositionQuery
            IMPL'/home/youse/apperception/eva/udfs/relative_position_query.py';
    """).df()

In [ ]:
### Set up SameVideo UDF
cursor.query(""" 
            CREATE UDF IF NOT EXISTS SameVideo
            IMPL'/home/youse/apperception/eva/udfs/same_video.py';
    """).df()

In [ ]:
# Certain attributes are made TEXTs due to issues Eva has with negative numbers
cursor.query("DROP TABLE IF EXISTS CameraConfigs;").df()
cursor.create_table("CameraConfigs", if_not_exists=True, columns="""
                cameraid TEXT(15),
                framenum INTEGER,
                cameratranslation NDARRAY FLOAT32(ANYDIM),
                camerarotation TEXT(100),
                cameraintrinsic NDARRAY FLOAT32(ANYDIM),
                egoheading TEXT(15),
                filename TEXT(30)
            """).df()


In [ ]:
### Load Data
from apperception.database import database
import pandas as pd

cursor.query("DROP TABLE IF EXISTS ObjectDetectionVideos;").df()

with open("scene-names.txt", 'r') as f:
    for sceneNumber in f.readlines():
        sceneNumber = sceneNumber.strip()
        # Load videos
        video_name = f"boston-seaport-scene-{sceneNumber}-CAM_FRONT.mp4"
        scene = f"scene-{sceneNumber}-CAM_FRONT"
        video_path = "/data/processed/full-dataset/trainval/videos/"
        cursor.load(file_regex=video_path + video_name, format="VIDEO", table_name='ObjectDetectionVideos').df()

        # Add camera configs
        result = database.execute(f"SELECT cameraId, frameNum, cameraTranslation, cameraRotation, cameraIntrinsic, egoHeading, filename FROM Cameras WHERE cameraId = '{scene}'")
        df = pd.DataFrame()
        for r in result:
            cameraId, frameNum, cameraTranslation, cameraRotation, cameraIntrinsic, egoHeading, filename = r
            cameraTranslation = list(cameraTranslation)
            # FrameNums in Eva are zero-indexed, so we subtract one before inserting
            cursor.query(f"""INSERT INTO CameraConfigs (cameraid, framenum, cameratranslation, camerarotation, cameraintrinsic, egoheading, filename) VALUES
                                        ('{cameraId}', {frameNum - 1}, {cameraTranslation}, '{cameraRotation}', {cameraIntrinsic}, '{egoHeading}', '{filename}');""").df()

        

In [ ]:
### Run query 1
start = time.time()

res1 = cursor.query("""
            SELECT framenum, id, cameraid, filename, name, egoheading, Yolo(data).labels, IntersectionQuery(LocationDetection(Yolo(data), MonodepthDetection(data).depth, cameratranslation, camerarotation, cameraintrinsic)) 
                FROM ObjectDetectionVideos JOIN CameraConfigs ON (id = framenum AND SameVideo(name, cameraid).issame)
                WHERE IntersectionQuery(LocationDetection(Yolo(data), MonodepthDetection(data).depth, cameratranslation, camerarotation, cameraintrinsic)).queryresult = 1
""").df()

end = time.time()
print("Query 1 Time:", format(end-start))

In [ ]:
### Run query 2
start = time.time()

res2 = cursor.query("""
            SELECT framenum, id, cameraid, filename, name, egoheading, Yolo(data).labels, IntersectionQuery(LocationDetection(Yolo(data), MonodepthDetection(data).depth, cameratranslation, camerarotation, cameraintrinsic)) 
                FROM ObjectDetectionVideos JOIN CameraConfigs ON (id = framenum AND SameVideo(name, cameraid).issame)
                WHERE RelativePositionQuery(LocationDetection(Yolo(data), MonodepthDetection(data).depth, cameratranslation, camerarotation, cameraintrinsic), cameratranslation, egoheading).queryresult = 1
""").df()

end = time.time()
print("Query 2 Time:", format(end-start))

In [ ]:
res2

In [ ]:
import cv2
import matplotlib.pyplot as plt


def displayFrame(row):
    prefix = "/data/raw/full-dataset/trainval/"
    # itemIds = ['scene-0757-CAM_FRONT_obj_44', 'scene-0757-CAM_FRONT_obj_23']
    frame_im = cv2.imread(prefix + row['cameraconfigs.filename'])
    plt.imshow(frame_im)


onlyTrue = res2[res2['intersectionquery.queryresult'] == True]
# res.apply(displayFrame, axis=1)
displayFrame(res2.iloc[7])
# onlyTrue.iloc[15]